In [ ]:
# import library

import torch
from torchvision import transforms
from PIL import Image
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import json
import urllib.request
import cv2
import torchvision.datasets as datasets 
from torch.utils.data import DataLoader,Dataset

In [ ]:
# We read the JSON data file 

with open("armanpaknia/Desktop/label.json", "r") as myfile :
    data = myfile.read()
data = json.loads(data)
len(data)

698

In [ ]:
#data haike label nakhordan o filde Label tuye file Json {} khali hast ro pak mikone

def Remove_empty_Label(data):
    all = []
    for x in range(len(data)):
        if data[x]["Label"] != {}:
            all.append(data[x])
    return all

all = Remove_empty_Label(data)

In [ ]:
import io
# ax haro az tuye file json load mikonim va zakhire mikonim dar Image
train = []
d=0
for x in range(len(all)):
    url = all[x].get("Labeled Data")
    url_response = urllib.request.urlopen(url)
    urllib.request.urlretrieve(url, "c:/Users/sinam/PycharmProjects/SoftwareProject/biene/train/ventilator/%d.jpg"%d)
    d += 1
    #img_array = np.array(bytearray(url_response.read()), dtype=np.uint8)
    #img = cv2.imdecode(img_array, -1)
    # img = io.BytesIO(url_response.read())
    # img = Image.open(img)
    # train.append(img)

    print(x)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
imgs = []
for x in range(len(all)):
    label = all[x]["Label"]["objects"]
    punkte = np.empty(0)
    for i in range(len(label)):

        for key in label[i]:
            if key == "line":
                punkt = label[i].get("line")
                for i in range(len(punkt)):
                    x = punkt[i].get("x")
                    y = punkt[i].get("y")
                    line = [x,y]
                    punkte = np.append(punkte, line)
    imgs.append(punkte)

points = np.array(imgs)

for i in range(points.shape[0]):

    points[i] = (points[i].reshape(int(points[i].shape[0] / 4),4))

In [ ]:
m = cv2.imread(files[100])

w, h = m.shape[0], m.shape[1]
print(w,h)

w_scale = 1000 / w
h_scale = 1000 / h
m = cv2.resize(m, (1000,1000))
pts = points[100]

for i in range(len(pts)):
    for j in range(4):
        if j == 0 or j == 2:
            pts[i][j] = pts[i][j] * h_scale
        else:
            pts[i][j] = pts[i][j] * w_scale
#pts = pts[i].reshape(-1,2)
print(pts)



pts = pts.reshape(-1,2)
print(len(pts))
print(pts)

for i in range(1, len(pts)+1):
    if i % 2 == 0:
        cv2.polylines(m, np.int32([pts[i-2:i]]) ,isClosed=False,color = (0, 0, 255), thickness=1)


cv2.imshow("sina", m)
cv2.waitKey(0)
cv2.destroyAllWindows()

3000 4000
[[312.7535     413.23       306.087      450.119     ]
 [200.4885     269.819      203.5005     310.783     ]
 [512.53675    375.04       507.115      407.16833333]
 [336.96275    272.245      341.46275    308.91166667]]
8
[[312.7535     413.23      ]
 [306.087      450.119     ]
 [200.4885     269.819     ]
 [203.5005     310.783     ]
 [512.53675    375.04      ]
 [507.115      407.16833333]
 [336.96275    272.245     ]
 [341.46275    308.91166667]]


In [ ]:
(files)

['c:/Users/sinam/PycharmProjects/SoftwareProject/biene/train\\ventilator\\0.jpg',
 'c:/Users/sinam/PycharmProjects/SoftwareProject/biene/train\\ventilator\\1.jpg',
 'c:/Users/sinam/PycharmProjects/SoftwareProject/biene/train\\ventilator\\2.jpg',
 'c:/Users/sinam/PycharmProjects/SoftwareProject/biene/train\\ventilator\\3.jpg',
 'c:/Users/sinam/PycharmProjects/SoftwareProject/biene/train\\ventilator\\4.jpg',
 'c:/Users/sinam/PycharmProjects/SoftwareProject/biene/train\\ventilator\\5.jpg',
 'c:/Users/sinam/PycharmProjects/SoftwareProject/biene/train\\ventilator\\6.jpg',
 'c:/Users/sinam/PycharmProjects/SoftwareProject/biene/train\\ventilator\\7.jpg',
 'c:/Users/sinam/PycharmProjects/SoftwareProject/biene/train\\ventilator\\8.jpg',
 'c:/Users/sinam/PycharmProjects/SoftwareProject/biene/train\\ventilator\\9.jpg',
 'c:/Users/sinam/PycharmProjects/SoftwareProject/biene/train\\ventilator\\10.jpg',
 'c:/Users/sinam/PycharmProjects/SoftwareProject/biene/train\\ventilator\\11.jpg',
 'c:/Users/sin

In [ ]:
from glob import glob
files =[]
DATA_DIR = "c:/Users/sinam/PycharmProjects/SoftwareProject/biene/"
train_dir = f"{DATA_DIR}train/"
valid_dir = f"{DATA_DIR}valid/"
for i in range( len(all)):
    for py in enumerate(glob(f'{DATA_DIR}train/*/%i.jpg'%i)):
        files.append(py[1])

In [ ]:
# from data_utils import create_validation_data
# import os
# if not os.path.exists(valid_dir):
#     create_validation_data(train_dir, valid_dir, split=0.2, ext='jpg')

100%|██████████| 89/89 [00:00<00:00, 428.00it/s]


445
89


In [ ]:
class BieneDataset(Dataset):
    def __init__(self, ds, points, sz=299):

        self.imgs = ds.imgs
        self.classes = ds.classes
        self.points = points
        self.sz = sz
        self.tfms = transforms.Compose([
            transforms.Resize((sz, sz)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

    def __getitem__(self, index):
        img, lbl = self.imgs[index]
        # get points
        pts = self.points[index]
        image = Image.open(img).convert('RGB')
        w, h = image.size
        w_scale = 1000 / w
        h_scale = 1000 / h

        # transformations
        image = self.tfms(image)

        for i in range(len(pts)):
            for j in range(4):
                if j == 0 or j == 2:
                    pts[i][j] = pts[i][j] * h_scale
                else:
                    pts[i][j] = pts[i][j] * w_scale

        # return image tensor, label tensor and bounding box tensor
        return image,lbl, torch.Tensor(pts)

    def __len__(self):
        return len(self.imgs)

631

In [ ]:
train_data = datasets.ImageFolder(train_dir)
train_ds = BieneDataset(train_data, points, sz=299)
train_dl = DataLoader(train_ds, batch_size=32, shuffle=True)

# valid_data = datasets.ImageFolder(valid_dir)
# valid_ds = BieneDataset(valid_data, points, sz=299)
# valid_dl = DataLoader(valid_ds, batch_size=32, shuffle=False)

In [ ]:
from vis_utils import imshow
import torchvision

dataiter = iter(train_dl)
imgs, lbl, bbs = next(dataiter)
img = torchvision.utils.make_grid(imgs, nrow=8)

plt.figure(figsize=(16,8))
imshow(img, title='A random batch of training data')

RuntimeError: stack expects each tensor to be equal size, but got [3, 4] at entry 0 and [10, 4] at entry 1

4
torch.Size([3, 299, 299])
0 : ventilator
[[ 417.50232  309.74924 1224.507   1352.997  ]
 [ 268.169    207.24925  812.507    954.997  ]
 [1248.507    668.997   1180.507    780.997  ]
 [2060.507   1122.997   2028.507   1220.997  ]]


NameError: name 'plot_bbox' is not defined

In [ ]:
# CNN
class Netz(nn.Module):
    def __init__(self):
        super(Netz, self).__init__()
        self.conv1 = nn.Conv2d(3, 100, kernel_size=5, padding=2)
        self.Batch1 = nn.BatchNorm2d(100)
        self.conv2 = nn.Conv2d(100, 50, kernel_size=5, padding=2)
        self.Batch2 = nn.BatchNorm2d(50)
        self.conv3 = nn.Conv2d(50, 30, kernel_size=5, padding=2)
        self.Batch3 = nn.BatchNorm2d(30)
        self.conv4 = nn.Conv2d(30, 20, kernel_size=5, padding=2)
        self.Batch4 = nn.BatchNorm2d(20)
        self.fc1 = nn.Linear(1280, 50)
        self.fc2 = nn.Linear(50, 3)

    def forward(self, x):

        x = self.conv1(x)
        x = self.Batch1(x)
        x = F.relu(x)

        x = self.conv2(x)
        x = self.Batch2(x)
        x = F.relu(x)

        x = self.conv3(x)
        x = self.Batch3(x)
        x = F.max_pool2d(x,2)
        x = F.relu(x)

        x = self.conv4(x)
        x = self.Batch4(x)
        x = F.max_pool2d(x,2)
        x = F.relu(x)

        #print(x.size())
        #quit()
        x = x.view(-1,1280)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

model = Netz()
model = model.cuda()
optimizer = optim.Adagrad(model.parameters(), lr= 0.001,weight_decay=0.05)
predict = []
predict_list_valid = []
predict_list_test = []
loss = []
def train(epoch):
    model.train()
    global predict_list_train

    for batch_id, (data, target) in enumerate(train_data):
        data = data.cuda()
        target = target.cuda()
        data = Variable(data)
        target = Variable(target)
        optimizer.zero_grad()
        out = model(data)
        criterion = F.nll_loss
        loss_train = criterion(out, target.view(-1).long().cuda())
        loss_train.backward()
        optimizer.step()



    print("Train Epoch : {} [ {} / {} ({:.0f}%)] \t Loss:{:.6f}".format(epoch, batch_id * 64,
                                                               len(data_x_train), 100 * batch_id / (len(train_data)-1),loss_train.data))
    loss.append(loss_train)
    return loss